In [ ]:
!pip install ndg-httpsclient

!pip install pyopenssl

!pip install pyasn1

In [ ]:
import requests
sess = requests.Session()
adapter = requests.adapters.HTTPAdapter(max_retries = 200)
sess.mount('http://', adapter)

#임원수 최종

In [ ]:
import pandas as pd
esg_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/esg_중심_기업목록.csv',encoding='UTF-8')
esg_df = esg_df.dropna(subset=['corp'])

In [ ]:
esg_df['corp_code'] = esg_df['corp'].astype(int).astype(str)

In [ ]:
# 80개씩 나누어서 각각의 DataFrame으로 저장
for i in range(0, len(esg_df), 100):
    globals()[f'esg_df_{i//100+1}'] = esg_df.iloc[i:i+100]

# 각 DataFrame 확인 (이름과 크기 출력)
for i in range(1, (len(esg_df) // 100) + 2):
    if f'esg_df_{i}' in globals():
        print(f"esg_df_{i}: {globals()[f'esg_df_{i}'].shape}")

esg_df_1: (100, 11)
esg_df_2: (100, 11)
esg_df_3: (100, 11)
esg_df_4: (100, 11)
esg_df_5: (100, 11)
esg_df_6: (100, 11)
esg_df_7: (100, 11)
esg_df_8: (100, 11)
esg_df_9: (100, 11)
esg_df_10: (100, 11)
esg_df_11: (28, 11)


In [ ]:
#전체 임원수 데이터 어떻게 생겼나 보기
# 공시정보 필수 Parameter로 가져오기
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
CORP_CODE = '01263022'
url='https://opendart.fss.or.kr/api/exctvSttus.json'
KEY="6fcbb95b62d25c42f004ec1a90e881ed91fbe938"
crtfc_key= KEY
YEAR="2023"
RPT_CODE="11011"

params={'crtfc_key': KEY
          , 'corp_code': CORP_CODE
          , 'bsns_year' : YEAR
          , 'reprt_code' : RPT_CODE}
results = requests.get(url, params=params, headers=headers,verify=False).json()
results
df = pd.DataFrame(results['list'])
df[['corp_code','corp_name',	'nm',	'sexdstn' ]]


,corp_code,corp_name,nm,sexdstn
0,01263022,BGF리테일,민승배,남
1,01263022,BGF리테일,홍정국,남
2,01263022,BGF리테일,백복현,남
3,01263022,BGF리테일,한명관,남
4,01263022,BGF리테일,임영철,남
5,01263022,BGF리테일,최자원,여
6,01263022,BGF리테일,신현상,남


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import urllib3
import warnings
import certifi
from IPython.display import display

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


# 공시정보 필수 Parameter로 가져오기
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
corp_list=esg_df_5['corp_code'].to_list() ####################

KEY="6fcbb95b62d25c42f004ec1a90e881ed91fbe938"
crtfc_key= KEY
YEAR="2023"
RPT_CODE="11011"
url='https://opendart.fss.or.kr/api/exctvSttus.json'

k=0
result_df = pd.DataFrame(columns=['corp_code','여성임원수', '남성임원수'])
for c in corp_list:
    k=k+1
    df_high = pd.DataFrame(columns=['corp_code','여성임원수', '남성임원수'])
    CORP_CODE = str(c).zfill(8)
    params={'crtfc_key': KEY
          , 'corp_code': CORP_CODE
          , 'bsns_year' : YEAR
          , 'reprt_code' : RPT_CODE}
    results = requests.get(url, params=params, headers=headers,verify=False).json()
    if results['status']=='000' :
        df = pd.DataFrame(results['list'])
        male_count = df[df['sexdstn'] == '남'].shape[0]
        female_count = df[df['sexdstn'] == '여'].shape[0]
        df_high = pd.DataFrame({
            'corp_code': [CORP_CODE],
            '여성임원수': [female_count],
            '남성임원수': [male_count] })

    result_df = pd.concat([result_df, df_high], axis=0, ignore_index=True)
    print(k)
result_df


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


,corp_code,여성임원수,남성임원수
0,00123718,0,5
1,00123772,0,48
2,00124027,0,10
3,00124106,0,25
4,00124151,0,7
...,...,...,...
82,00134176,1,6
83,00585219,1,16
84,00422895,0,5
85,00415628,1,9


In [ ]:
result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/임원수_df/df_401_500.csv',index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import urllib3
import warnings
import certifi
from IPython.display import display

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
import pandas as pd
import os

# 특정 디렉토리 경로 지정
directory = '/content/drive/MyDrive/Colab Notebooks/임원수_df'  # '/content/' 부분은 Colab의 기본 경로입니다.

# 디렉토리 내의 파일 목록 가져오기
file_list = os.listdir(directory)

# CSV 파일만 필터링
csv_files = [file for file in file_list if file.endswith('.csv')]

# CSV 파일들을 DataFrame으로 읽어오기
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# DataFrame들이 존재할 때만 concat 수행
if dfs:
    df_combined = pd.concat(dfs, ignore_index=True)
else:
    print("No DataFrames to concatenate.")


In [ ]:
df_combined.to_csv('/content/drive/MyDrive/Colab Notebooks/임원수_df/df_executive.csv', index=False)

In [ ]:
df_executive = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/임원수_df/df_executive.csv', encoding = 'UTF-8')

In [ ]:
df_executive = df_executive.to_csv('/content/drive/MyDrive/Colab Notebooks/임원수_df/df_executive.csv', index=False, encoding='utf-8-sig')